# Data exploration of input videos

In [8]:
!ffprobe -hide_banner input_files/Cosmos_War_of_the_Planets.mp4

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'input_files/Cosmos_War_of_the_Planets.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: mp42mp41
    creation_time   : 2021-08-02T19:15:48.000000Z
  Duration: 00:00:20.05, start: 0.000000, bitrate: 3309 kb/s
  Stream #0:0[0x1](eng): Video: h264 (Main) (avc1 / 0x31637661), yuv420p(progressive), 628x354 [SAR 1:1 DAR 314:177], 2989 kb/s, 29.97 fps, 29.97 tbr, 30k tbn (default)
    Metadata:
      creation_time   : 2021-08-02T19:15:48.000000Z
      handler_name    : ?Mainconcept Video Media Handler
      vendor_id       : [0][0][0][0]
      encoder         : AVC Coding
  Stream #0:1[0x2](eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 317 kb/s (default)
    Metadata:
      creation_time   : 2021-08-02T19:15:48.000000Z
      handler_name    : #Mainconcept MP4 Sound Media Handler
      vendor_id       : [0][0][0][0]


In [9]:
from IPython.display import Video
Video("input_files/Cosmos_War_of_the_Planets.mp4")

# Video analysis and report generation 

In [10]:
EXPECTED_FORMAT = {
    "container": "mp4",
    "video_codec": "h.264",
    "audio_codec": "aac",
    "frame_rate": "25",
    "aspect_ratio": "16:9",
    "resolution": "640x360",
    "video_br": "2 – 5 Mb/s",
    "audio_br": "up to 256 kb/s",
    "audio_channels": "stereo"
}

In [12]:
VIDEO_FILENAME = "input_files/Cosmos_War_of_the_Planets.mp4"

In [14]:
!ffprobe -hide_banner VIDEO_FILENAME

VIDEO_FILENAME: No such file or directory


Function that runs ffprobe command inside python and returns its output inspired by: https://stackoverflow.com/a/9896732

In [24]:
import os, sys, subprocess, shlex, re
import json
from subprocess import call

def probe_file(filename):
    cmnd = ['ffprobe', "-print_format", "json", '-show_format', '-pretty', '-loglevel', 'quiet', filename]
    p = subprocess.Popen(cmnd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err =  p.communicate()
    if err:
        print ("========= error ========")
        print(err)
        raise Exception(err)
    return json.loads(out)

def parse_probe(probe_r):
    # TODO: EXTRACT NEEDED INFO FROM FFPROBE OUTPUT HERE
    r = {}
    return r

result = probe_file(VIDEO_FILENAME)
video_format = parse_probe(result)

In [25]:
result

{'format': {'filename': 'input_files/Cosmos_War_of_the_Planets.mp4',
  'nb_streams': 2,
  'nb_programs': 0,
  'format_name': 'mov,mp4,m4a,3gp,3g2,mj2',
  'format_long_name': 'QuickTime / MOV',
  'start_time': '0:00:00.000000',
  'duration': '0:00:20.053333',
  'size': '7.911813 Mibyte',
  'bit_rate': '3.309629 Mbit/s',
  'probe_score': 100,
  'tags': {'major_brand': 'mp42',
   'minor_version': '0',
   'compatible_brands': 'mp42mp41',
   'creation_time': '2021-08-02T19:15:48.000000Z'}}}